In [13]:
from firecrown.models.cluster.abundance import ClusterAbundance
from firecrown.models.cluster.deltasigma import ClusterDeltaSigma
import pyccl as ccl
import numpy as np
from firecrown.models.cluster.recipes.murata_binned_spec_z_deltasigma import MurataBinnedSpecZDeltaSigmaRecipe as MDS
import sacc
from firecrown.modeling_tools import ModelingTools
from firecrown.models.cluster.properties import ClusterProperty
from firecrown.likelihood.binned_cluster_number_counts_deltasigma import BinnedClusterDeltaSigma

# Firecrown Tutorial

## 1. Introduction
Firecrown is a framework to run likelihoods. It connects theory predictions to `cosmosis`, a sampler framework, enabling MCMC runs.  

There are several likelihood options built into Firecrown. Generally, we just need to provide:
- A likelihood Python function
- Some configuration files  

In this tutorial, we will focus on:
- How to run the cluster examples
- How the prediction module works
- How to structure a Python likelihood for Firecrown
- How to obtain the theory prediction in a Python file


## 2. How to run and which files are needed
To run Firecrown, we need **three different files**:

1. **Likelihood file** (e.g. `cluster_redshift_richness_deltasigma.py`)  
2. **Cosmosis configuration file** (e.g. [cluster_counts_mean_mass_redshift_richness_shear.ini](https://github.com/LSSTDESC/firecrown/blob/master/examples/cluster_number_counts/cluster_counts_mean_mass_redshift_richness_shear.ini))  
3. **Cosmosis parameters file** – defines cosmological and sampling parameters (e.g. [cluster_richness_values_deltasigma.ini](https://github.com/LSSTDESC/firecrown/blob/master/examples/cluster_number_counts/cluster_richness_values_deltasigma.ini))  

## 3. Likelihood file
The likelihood file must define the method `build_likelihood`, which returns a Firecrown likelihood and modeling tools.

For example, in `cluster_redshift_richness_deltasigma.py`:

```
def build_likelihood(
    build_parameters: NamedParameters,
) -> tuple[Likelihood, ModelingTools]:
    """Builds the likelihood for Firecrown."""
    # Pull params for the likelihood from build_parameters
    average_on = ClusterProperty.NONE
    if build_parameters.get_bool("use_cluster_counts", True):
        average_on |= ClusterProperty.COUNTS
    if build_parameters.get_bool("use_mean_log_mass", True):
        average_on |= ClusterProperty.MASS
    if build_parameters.get_bool("use_mean_deltasigma", True):
        average_on |= ClusterProperty.DELTASIGMA

    survey_name = "numcosmo_simulated_redshift_richness_deltasigma"
    likelihood = ConstGaussian(
        [
            BinnedClusterNumberCounts(
                average_on, survey_name, MurataBinnedSpecZRecipe()
            ),
            BinnedClusterDeltaSigma(
                average_on, survey_name, MurataBinnedSpecZDeltaSigmaRecipe()
            ),
        ]
    )

    # Read in sacc data
    sacc_file_nm = "cluster_redshift_richness_deltasigma_sacc_data.fits"
    sacc_path = os.path.expanduser(
        os.path.expandvars("${FIRECROWN_DIR}/examples/cluster_number_counts/")
    )
    sacc_data = sacc.Sacc.load_fits(os.path.join(sacc_path, sacc_file_nm))
    likelihood.read(sacc_data)

    cluster_abundance = get_cluster_abundance()
    cluster_deltasigma = get_cluster_deltasigma()

    modeling_tools = ModelingTools(
        cluster_abundance=cluster_abundance, cluster_deltasigma=cluster_deltasigma
    )

    return likelihood, modeling_tools
```

Key points:
- The likelihood must **read the data** from a SACC file.  
- Firecrown provides helpers to interpret the data.  
- The likelihood must also include **statistics that compute the theoretical prediction** (e.g. `BinnedClusterNumberCounts`, `BinnedClusterDeltaSigma`).  

## 4. Cosmosis Configuration Files

In the Cosmosis files, we define the sampler to be used, the configurations for the sampler, and which likelihood file we will use. It is basically to set all the paths and recommended files in place. Let us look at `examples/cluster_number_counts/cluster_counts_mean_mass_redshift_richness_shear.ini`:



```
[runtime]
sampler = test
resume = T
root = ${PWD}
```

The test sampler runs just one iteration. It is useful to see if everything is set correctly. When running an MCMC, one must change the sampler option.

In the pipeline section, we set the path to the parameters file, in this case `cluster_richness_values_deltasigma.ini`, and the modules we want to use:

```
[pipeline]
modules = consistency camb firecrown_likelihood
values = cluster_richness_values_deltasigma.ini
likelihoods = firecrown
quiet = T
debug = T
timing = T
```

Finally, in the Firecrown likelihood section, we set parameters that will be used in our likelihood file above. You can see that these parameters were accessed with `build_parameters` in the likelihood file. The `sampling_parameters_sections` are defined in the parameters file cited above. The cosmological parameters are already expected by the file, so if your statistics have some unique parameters to be varied, you have to specify them in a new section and include this section here.


```
[firecrown_likelihood]
;; Fix this to use an environment variable to find the files.
;; Set FIRECROWN_DIR to the base of the firecrown installation (or build, if you haven't
;; installed it)
file = /global/cfs/projectdirs/lsst/groups/CL/cl_pipeline_project/conda_envs/firecrown_clp/lib/python3.13/site-packages/firecrown/connector/cosmosis/likelihood.py
likelihood_source = cluster_redshift_richness_deltasigma.py
sampling_parameters_sections = firecrown_number_counts
use_cluster_counts = True
use_mean_log_mass = False
use_mean_deltasigma = True
```

```
[test]
fatal_errors = T
save_dir = output_counts_mean_mass

[metropolis]
samples = 1000
nsteps = 1
```


## 5. The parameters configuration file
In the parameter configuration file is where we set our cosmological parameters plus the parameters that need to be varied in our code. Let us take a look at `cluster_richness_values_deltasigma.ini`

```
; Parameters and data in CosmoSIS are organized into sections
; so we can easily see what they mean.
; There is only one section in this case, called cosmological_parameters
[cosmological_parameters]

; These are the only cosmological parameters being varied.
omega_c = 0.1552 0.22 0.3552
# We are choosing to use a flat prior in sigma_8.
# To choose a float prior in A_s, remove the specification
# of a prior for sigma_8 and replace it with the
# desired range for A_s
sigma_8 = 0.7 0.800 0.9
; The following parameters are set, but not varied.
;
omega_k = 0.0
omega_b = 0.0448
tau = 0.08
n_s = 0.963
h0 = 0.71
w = -1.0
wa = 0.0

[firecrown_number_counts]
; These are the firecrown likelihood parameters.
; These parameters are used to set the richness-mass
; proxy relation using the data from cluster number counts.
;
; The following parameters can be fixed in the same way as the above
; cosmological parameters if needed.
mu_p0 = 3.19
mu_p1 = 0.868
mu_p2 = -0.3
sigma_p0 = 0.33
sigma_p1 = -0.034 
sigma_p2 = 0.0
cluster_conc = 4.
```

The cosmological parameters section is always present, while for this specific example, we define also the firecrown number counts section with the parameters we want to vary or set in our statiscs. A single value means that this will be the parameter set for this parameter. 3 values mean that the value will be varied between the left and right and start in the midle, with flat priors.

## 6. Running Firecrown clusters

Once all files are set, run in the terminal:

```bash
cosmosis cluster_counts_mean_mass_redshift_richness_shear.ini
```

⚠️ Use this only with the **test sampler**.  
For MCMC chains, submit as a **slurm job**.  

After running:

```bash
cosmosis-postprocess -o output/cluster_number_counts.txt --burn 0.3
```

This generates plots and statistics using the `cosmosis-postprocess` package.

## 7. Cluster modeling in Firecrown
Now we will discuss a bit of the modeling theoretical prediction code that is present in firecrown. So the modeling code consists on some files:

- `abundance.py` → cluster abundance object  
- `deltasigma.py` → tangential shear predictions

These two files define the cluster abundance object and the predictions for the shear and cluster abundance below we will see how it works. Note that: the mass information for clusters are in log10. To update the cosmology we have to use the update ingredients that comes from the updatable object

Another files I will not get into detail are:
- `cluster_data.py`, `deltasigma_data.py`, `abundance_data.py` → data handling (especially SACC files)  
- `kernel.py` → selection functions (completeness, purity)  
- `mass_proxy.py` → mass proxy

Let us see the cluster objects.

In [1]:
hmf = ccl.halos.MassFuncDespali16()
min_mass, max_mass = 13., 16.
min_z, max_z = 0.2, 0.8
cluster_deltasigma = ClusterDeltaSigma((min_mass, max_mass), (min_z, max_z), hmf)
cluster_abundance = ClusterAbundance((min_mass, max_mass), (min_z, max_z), hmf)
cosmo_ccl = ccl.Cosmology(
Omega_c=0.2052,
Omega_b=0.0448,
h=0.71,
n_s=0.963,
sigma8=0.8,
Omega_k=0.0,
Neff=3.044,
m_nu=0.0,
w0=-1.0,
wa=0.0,
T_CMB=2.7255
)
cluster_abundance.update_ingredients(cosmo_ccl)
cluster_deltasigma.update_ingredients(cosmo_ccl)
print(cluster_abundance.mass_function(mass=np.array([13.0]),z=np.array([1.0])))
print(cluster_deltasigma.delta_sigma(log_mass=np.array([13.0]),z=np.array([1.0]), radius_center=2.0))


[0.00027175]
[1.9362217e+12]


## 8. Recipes
Recipes put everything together into integrals used in statistics.  

Example: [`murata_binned_spec_z_deltasigma.py`](https://github.com/LSSTDESC/firecrown/blob/master/firecrown/models/cluster/recipes/murata_binned_spec_z_deltasigma.py)

They define:
- Redshift distribution  
- Mass distribution  
- Integrals for predictions  

Key method:

```
def get_theory_prediction(
    self,
    cluster_theory: ClusterDeltaSigma,
    average_on: None | ClusterProperty = None,
) -> Callable:
    def theory_prediction(
        mass, z, mass_proxy_limits, sky_area, radius_center,
    ):
        prediction = (
            cluster_theory.comoving_volume(z, sky_area)
            * cluster_theory.mass_function(mass, z)
            * self.redshift_distribution.distribution()
            * self.mass_distribution.distribution(mass, z, mass_proxy_limits)
        )
        if average_on is None:
            raise ValueError("The property should be ClusterProperty.DELTASIGMA.")

        if ClusterProperty.DELTASIGMA in ClusterProperty:
            prediction *= cluster_theory.delta_sigma(mass, z, radius_center)
        return prediction

    return theory_prediction
```

The final evaluation integrates this function:

```
def evaluate_theory_prediction(self, cluster_theory, this_bin, sky_area, average_on=None):
    self.integrator.integral_bounds = [
        (cluster_theory.min_mass, cluster_theory.max_mass),
        this_bin.z_edges,
    ]
    radius_center = this_bin.radius_center
    self.integrator.extra_args = np.array(
        [*this_bin.mass_proxy_edges, sky_area, radius_center]
    )
    theory_prediction = self.get_theory_prediction(cluster_theory, average_on)
    prediction_wrapper = self.get_function_to_integrate(theory_prediction)
    return self.integrator.integrate(prediction_wrapper)
```

In [4]:
def counts_deltasigma_prediction_from_sacc(path, survey_nm, pivot_mass, pivot_redshift, mu_p0, mu_p1, mu_p2, sigma_p0, sigma_p1, sigma_p2, mass_parameter=False):
    s_read = sacc.Sacc.load_fits(path)

    
    hmf = ccl.halos.MassFuncDespali16()
    min_mass, max_mass = 13., 16.
    min_z, max_z = 0.2, 0.8
    cluster_deltasigma = ClusterDeltaSigma((min_mass, max_mass), (min_z, max_z), hmf)
    cluster_abundance = ClusterAbundance((min_mass, max_mass), (min_z, max_z), hmf)
    cosmo_ccl = ccl.Cosmology(
    Omega_c=0.2052,
    Omega_b=0.0448,
    h=0.71,
    n_s=0.963,
    sigma8=0.8,
    Omega_k=0.0,
    Neff=3.044,
    m_nu=0.0,
    w0=-1.0,
    wa=0.0,
    T_CMB=2.7255
    )
    cluster_abundance.update_ingredients(cosmo_ccl)
    cluster_deltasigma.update_ingredients(cosmo_ccl)
    
    modeling_tools = ModelingTools(cluster_abundance = cluster_abundance, cluster_deltasigma=cluster_deltasigma)
    mds = MDS()
    mds.mass_distribution.pivot_mass = np.log(10**pivot_mass)
    mds.mass_distribution.pivot_redshift = pivot_redshift
    mds.mass_distribution.log1p_pivot_redshift = np.log1p(pivot_redshift)
    mds.mass_distribution.mu_p0 = mu_p0
    mds.mass_distribution.mu_p1 = mu_p1
    mds.mass_distribution.mu_p2 = mu_p2
    mds.mass_distribution.sigma_p0 = sigma_p0
    mds.mass_distribution.sigma_p1 = sigma_p1
    mds.mass_distribution.sigma_p2 = sigma_p2

    average_on = ClusterProperty.DELTASIGMA
    if mass_parameter:
        average_on |= ClusterProperty.MASS
    bin_cl_theory = BinnedClusterDeltaSigma(average_on, survey_nm, mds)
    bin_cl_theory.read(s_read)
    cluster_abundance.update_ingredients(cosmo_ccl)

    prediction = bin_cl_theory._compute_theory_vector(modeling_tools)
    data = bin_cl_theory.data_vector
    return data, prediction

Since we are defining the prediction function from the recipe in a python file, we had to set several parameter by hand that would be set by the cosmosis configuration files. We are also running some read and update methods that would be done by the statics object when passing this recipe. Now let us make a function call

In [16]:
firecrown_prediction = counts_deltasigma_prediction_from_sacc("cluster_redshift_richness_deltasigma_sacc_data.fits", "numcosmo_simulated_redshift_richness_deltasigma", 13.5, 0.5, 3.19, 2.0, 0.04, 0.5, 0.03, 0.01)
print(firecrown_prediction)

(DataVector([7.69456258e+13, 4.88340539e+13, 2.81903346e+13,
            1.50059006e+13, 7.47930512e+12, 3.53993773e+12,
            1.00140347e+14, 6.55628245e+13, 3.89231822e+13,
            2.12128026e+13, 1.07748291e+13, 5.17584313e+12,
            1.34220492e+14, 9.08611966e+13, 5.56101838e+13,
            3.10956478e+13, 1.61233697e+13, 7.87047257e+12,
            1.77097201e+14, 1.24101498e+14, 7.84585303e+13,
            4.51150602e+13, 2.39296213e+13, 1.18910106e+13,
            2.11761296e+14, 1.51156091e+14, 9.72921728e+13,
            5.68390432e+13, 3.05459884e+13, 1.53374038e+13,
            7.71643036e+13, 4.84365434e+13, 2.76951720e+13,
            1.46279710e+13, 7.24643301e+12, 3.41352598e+12,
            1.05885791e+14, 6.87778375e+13, 4.05337698e+13,
            2.19516568e+13, 1.10926783e+13, 5.30669208e+12,
            1.45397707e+14, 9.80214669e+13, 5.97580891e+13,
            3.33013800e+13, 1.72188395e+13, 8.38660449e+12,
            1.75263485e+14, 1.20810536e

## 10. SACC data
SACC stores data with **tags for each data type**:
- Cluster number counts: 3 tracers (`richness`, `redshift`, `survey`)  
- Shear: 4 tracers (the same 3 plus `radius center`)  

Each data point must be added with its tracer information.  

Example: opening a SACC file to inspect contents.  


In [20]:
import pprint
s = sacc.Sacc.load_fits("cluster_redshift_richness_deltasigma_sacc_data.fits")
pprint.pprint(s.tracers)
print(f"\n\n\n")
pprint.pprint(s.data)

{np.str_('bin_radius_0'): <sacc.tracers.BinRadiusTracer object at 0x7fa82c42f9d0>,
 np.str_('bin_radius_1'): <sacc.tracers.BinRadiusTracer object at 0x7fa82c42f7d0>,
 np.str_('bin_radius_2'): <sacc.tracers.BinRadiusTracer object at 0x7fa82c42ccd0>,
 np.str_('bin_radius_3'): <sacc.tracers.BinRadiusTracer object at 0x7fa82c42de50>,
 np.str_('bin_radius_4'): <sacc.tracers.BinRadiusTracer object at 0x7fa82c42c4d0>,
 np.str_('bin_radius_5'): <sacc.tracers.BinRadiusTracer object at 0x7fa82c4b40d0>,
 np.str_('bin_z_0'): <sacc.tracers.BinZTracer object at 0x7fa86aa102f0>,
 np.str_('bin_z_1'): <sacc.tracers.BinZTracer object at 0x7fa83423b930>,
 np.str_('bin_z_2'): <sacc.tracers.BinZTracer object at 0x7fa8358316a0>,
 np.str_('bin_z_3'): <sacc.tracers.BinZTracer object at 0x7fa835831b70>,
 np.str_('numcosmo_simulated_redshift_richness_deltasigma'): <sacc.tracers.SurveyTracer object at 0x7fa82bddd950>,
 np.str_('rich_0'): <sacc.tracers.BinRichnessTracer object at 0x7fa835831940>,
 np.str_('rich_1